# Overview

We will introduce the way we formalize the organization of an experiment in a *design matrix*.

# Goals

* Compute averages of activity for different types of events in a localizer experiment.
* Understand how responses to stimuli evolve over time

In [ ]:
# Load necessary libraries
import matplotlib.pyplot as plt
import cortex as cx
import neurods as nds
import numpy as np
from scipy.stats import zscore
import os

In [ ]:
# Update neurods library
nds.io.update_neurods()
# Update figures
os.symlink('/home/shared/cogneuro-connector/data/Week09_EventAvg_ImpulseResponse/figures/','figures')

In [ ]:
# Matplotlib defaults
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

# Load data

Same as previously: load using nibabel, use get_data() method of the nibabel data object, transpose resulting data array, and zscore the data. For now, we will treat this as a generic data set (next week, we will learn more about the experiment that generated this data as we begin to actually analyze the data).

In [ ]:
### TEACHER INFO


In [ ]:
sub, xfm = 'S2', 'S2_category_auto'
basedir = os.path.join(nds.io.data_list['fmri'], 'categories')
fname = os.path.join(basedir, 'sub01_categories1_1.nii.gz')
mask = cx.db.get_mask(sub, xfm, type='cortical')
data = nds.fmri.load_data(fname, mask=mask, standardize=True)
print("Data dimensions are: ", data.shape)

In [ ]:
### TEACHER INFO


## Breakout session
> Two weeks ago, we made a plot of a single slice over time. Can you make a plot like that using nds.viz.slice_3d_array? (Plot the 8th slice over time). 

In [ ]:
### STUDENT ANSWER


# Quick pycortex review

In [ ]:
# Create a volume
vkw = dict(mask=mask, cmap='RdBu_r', vmin=-3, vmax=3)
vol = cx.Volume(data[0], sub, xfm, **vkw)
cx.webgl.show(vol)

In [ ]:
# Show a flamtap
# (If you get warnings about a module called shapely, ignore them; they are not important.)
_ = cx.quickflat.make_figure(vol)

## Load description of experiment 
The experiment we have been working with is a *localizer* experiment. It is designed to find areas of the brain that respond to particular visual categories of objects: faces, bodies, and places. It also reveals areas that respond more to objects than to scrambled versions of the same objects. This experiment is a simple replication of past work, and is commonly done as a first step to locate (or localize) a region of interest for further analysis in a subsequent experiment.

For the localizer experiment, images from each category were presented in a block design. This means that images from the one category were shown one after another for a "block" of 20 seconds (10 TRs), followed by images from another category for a block of 20 seconds, and so on.

<img src="figures/CategoryLocalizerDesign.001.png" style="height: 400px;">

To analyze the data from this experiment at all, we need to know when the blocks for each category (faces, bodies, places, objects, and scrambled objects) began and ended. This information is stored in a *design matrix*, which we load below.

In [ ]:
design = np.load(os.path.join(basedir, 'experiment_design.npz'))
print('Experiment design variables: ', sorted(design.keys()))

In [ ]:
conditions = design['conditions'].tolist()
print('Conditions: ', conditions)
design_run1 = design['run1']
print('Design shape: ', design_run1.shape)

It's often useful to show a design matrix as an image:

In [ ]:
_ = plt.imshow(design_run1.T)

## Breakout session
> What are the dimensions here? Label the axes on the figure above!

In [ ]:
### STUDENT ANSWER


# Find condition onsets
Last week for homework you wrote a function to compute event-related averages of data, given condition onset times. Here's a good version of such a function:

In [ ]:
nds.fmri.compute_event_avg??

Here, we don't have explicit times, only logical indices for which timepoints belong to which conditions - so we need to compute when the condition onsets were to use our function, or we need to write a new function! Let's stick with the old one, as we'll use it later, and just find the condition onsets.

> Find the onsets for each condition! And make your code into a re-usable function to find onsets

In [ ]:
### STUDENT ANSWER


SO: Compute event averages for one condition!

In [ ]:
cond = 'face'
idx = conditions.index(cond)
cond_onsets = get_onsets(design_run1, idx)
n_time_points = 10
cond_avg = nds.fmri.compute_event_avg(data, cond_onsets, n_time_points)
print("Condition average (cond_avg) shape:", cond_avg.shape)

In [ ]:
# Show a bunch of flatmaps in sequence
vkw = dict(mask=mask, cmap='RdBu_r', vmin=-2, vmax=2)
for n, ca in enumerate(cond_avg):
    cx.quickflat.make_figure(cx.Volume(ca, sub, xfm, **vkw), height=300)
    plt.title('{n} TRs from onset'.format(n=n))

In [ ]:
# Show a movie of the event average timecourse
vkw = dict(cmap='RdBu_r', vmin=-2, vmax=2)
cx.webgl.show(cx.Volume(cond_avg, sub, xfm, **vkw))

## Discussion
How does the event average change over time? Why do you think this is? 

## Breakout session
> Do this for all conditions; make your code compact, if you can! Keep your results in a dictionary called event_avg

In [ ]:
### STUDENT ANSWER


## Compute event averages with MOAR DATA

NOTE: Unclear how loading 3 sessions will play with memory limits!

In [ ]:
# Get MORE DATA!
fname = os.path.join(basedir, 'sub01_categories1_{n}.nii.gz')
data_3runs = np.vstack([nds.fmri.load_data(fname.format(n=n), mask=mask, standardize=True) for n in [1,2,3]])
print("Dimensions of the data: ", data_3runs.shape)

In [ ]:
design_3runs = np.vstack([design['run{n}'.format(n=n)] for n in [1,2,3]])
print("Dimensions of the design matrix: ",design_3runs.shape)

In [ ]:
### STUDENT ANSWER


In [ ]:
# again, perhaps overkill
for n, tr in enumerate(event_avg['face']): 
    cx.quickflat.make_figure(cx.Volume(tr, sub, xfm, **vkw), height=300, )
    plt.title('{n} TRs from onset'.format(n=n))

---